In [3]:
# Import modules
import torch
import pandas as pd

In [4]:
def set_device():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    if torch.cuda.is_available():
        print(f'Using device: {device}')
        print(f'GPU: {torch.cuda.get_device_name(0)}')
    else:
        print(f'Using device: {device}')

    return device

In [5]:
def norm_data(name):
    df = pd.read_csv(name)
    range_mm={
        'V': {'min':df['V'].min()*0.8, 'max': df['V'].max()*1.2},
        'E': {'min':df['E'].min()*0.8, 'max': df['E'].max()*1.2},
        'VF': {'min':df['VF'].min()*0.8, 'max': df['VF'].max()*1.2},
        'VA': {'min':df['VA'].min()*0.8, 'max': df['VA'].max()*1.2},
        'VB': {'min':df['VB'].min()*0.8, 'max': df['VB'].max()*1.2},
        'CFLA': {'min':0, 'max': df['CFLA'].max()*1.2},
        'CALA': {'min':0, 'max': df['CALA'].max()*1.2},
        'CBLA': {'min':0, 'max': df['CBLA'].max()*1.2},
        'CFK': {'min':0, 'max': df['CFK'].max()*1.2},
        'CAK': {'min':0, 'max': df['CAK'].max()*1.2},
        'CBK': {'min':0, 'max': df['CBK'].max()*1.2},
        'I': {'min':0, 'max': df['I'].max()*1.2},

    }
    
    return range_mm

In [6]:
if __name__ == '__main__':
    device = set_device()
    rangemm = norm_data('BMED_DATA_AG.csv')
    print(rangemm)

Using device: cuda
GPU: NVIDIA GeForce RTX 4050 Laptop GPU
{'V': {'min': np.float64(8.0), 'max': np.float64(42.0)}, 'E': {'min': np.float64(0.2), 'max': np.float64(1.2)}, 'VF': {'min': np.float64(0.1899317879530128), 'max': np.float64(1.8489795918367347)}, 'VA': {'min': np.float64(0.5632653061224491), 'max': np.float64(1.8274111675126905)}, 'VB': {'min': np.float64(0.7287128712871287), 'max': np.float64(7.309644670050763)}, 'CFLA': {'min': 0, 'max': np.float64(4.1209425)}, 'CALA': {'min': 0, 'max': np.float64(2.55057791291714)}, 'CBLA': {'min': 0, 'max': np.float64(0.09083684777202455)}, 'CFK': {'min': 0, 'max': np.float64(7.073639999999999)}, 'CAK': {'min': 0, 'max': np.float64(0.07949414894162436)}, 'CBK': {'min': 0, 'max': np.float64(3.158869885325891)}, 'I': {'min': 0, 'max': np.float64(5.28)}}


In [ ]:
# 데이터 로드 및 확인
df = pd.read_csv('BMED_DATA_AG.csv')
print(f"데이터 형태: {df.shape}")
print(f"컬럼들: {df.columns.tolist()}")
print(df.head())